# Nested Interval Property

This is a proof of the nested interval property of real numbers. Original proof from Understanding Analysis by Stephen Abbott.

In [1]:
from pylogic import *

In [ ]:
settings["PYTHON_OPS_RETURN_PROPS"] = True

a = Sequence("a", real=True)
b = Sequence("b", real=True)

I = Sequence(
    "I",
    set_=True,
    nth_term=lambda n: interval(
        [a[n], b[n]],
    ),
)

# The proof esssentially goes as follows:

# 1. A is nonempty, since a0 is in A
# 2. bk is an upper bound for sequence a, and therefore set A = {a[n] | n in Naturals}
# 3. A has a least upper bound, lub_A
# 4. Given any k, a[k] <= lub_A <= b[k], so lub_A is in each interval I[k]

n = Variable("n")
k0 = Variable("k0")
k1 = Variable("k1")

forall_n_in_N_an_leq_bn = ForallInSet(n, Naturals, a[n] <= b[n]).assume()
forall_k_and_n_in_N_k_leq_n_implies_ak_leq_an_and_bn_leq_bk = ForallInSet(
    k0,
    Naturals,
    ForallInSet(
        k1, Naturals, if_(k0 <= k1).then((a[k0] <= a[k1]).and_(b[k1] <= b[k0]))
    ),
).assume()

A = SeqSet(a)

# Part 1: A is nonempty
# proving that A is nonempty by showing that a0 is in A
a0 = a[zero]
a0_in_A = a0.is_in(A).by_inspection()
A_nonempty = a0_in_A.thus_not_empty()


# Part 2: bk is an upper bound for set A
# here, we prove that bk is an upper bound for A, so that we can show that A has a least upper bound
# proving that an <= bk for all n and k in Naturals
with AssumptionsContext() as ctx:
    k = ctx_var("k", natural=True)
    bk = b[k]
    ak = a[k]
    Ik = I[k]

    # given any x in A, we show that x <= bk
    with AssumptionsContext() as x_ctx:
        x = ctx_var("x")
        x_in_A = x.is_in(A).assume()
        n, (_, x_eq_an) = x_in_A.thus_predicate().rename_variable("n")

        # we do a proof by cases

        # case 1: n <= k
        with AssumptionsContext() as case_1:
            n_leq_k = assume(n <= k)
            an_leq_ak = forall_k_and_n_in_N_k_leq_n_implies_ak_leq_an_and_bn_leq_bk(
                n, k
            )(n_leq_k)[0]
            ak_leq_bk = forall_n_in_N_an_leq_bn(k)
            an_leq_bk = an_leq_ak.transitive(ak_leq_bk)
            conclude(an_leq_bk)
        impl1 = case_1.get_proven()[0]
        # print(k.is_natural)

        with AssumptionsContext() as case_2:
            # case 2: k <= n
            k_leq_n = assume(k <= n)
            an_leq_bn = forall_n_in_N_an_leq_bn(n)
            bn_leq_bk = forall_k_and_n_in_N_k_leq_n_implies_ak_leq_an_and_bn_leq_bk(k, n)(k_leq_n)[1]
            an_leq_bk = an_leq_bn.transitive(bn_leq_bk)
            conclude(an_leq_bk)
        impl2 = case_2.get_proven()[0]

        # strongly connected: for all naturals a,b, a<=b or b<=a
        leq_is_strongly_connected = Naturals.order_is_strongly_connected(n, k)
        an_leq_bk = leq_is_strongly_connected.by_cases(impl1, impl2)
        conclude(an_leq_bk.substitute("left", x_eq_an))

    forall_x_in_A_x_leq_bk = x_ctx.get_proven()[0] # this was Part 2 goal
    # therefore, A has an upper bound
    exists_real_ub_for_A = forall_x_in_A_x_leq_bk.thus_there_exists("ub", bk, Reals)


    # Part 3: A has a least upper bound
    # by the completeness axiom, A has a least upper bound
    # we extract it and get its properties
    A_has_lub = Reals.bounded_above_has_lub(A)(A_nonempty, exists_real_ub_for_A)
    lub_A, lub_A_is_ub_and_least = A_has_lub.extract()
    lub_A_is_real = lub_A_is_ub_and_least[0]
    lub_A_is_ub = lub_A_is_ub_and_least[1]
    lub_A_is_least = lub_A_is_ub_and_least[2]

    # Part 4: a[k] <= lub_A <= b[k]
    # therefore lub_A is in I[k] = [a[k], b[k]]
    ak_leq_lub = lub_A_is_ub(ak)
    if_bk_ub_then_lub_leq_bk = lub_A_is_least(bk)
    lub_leq_bk = forall_x_in_A_x_leq_bk.modus_ponens(if_bk_ub_then_lub_leq_bk)

    lub_in_interval_k = lub_A.is_in(interval([ak, bk])).by_predicate(lub_A_is_real.and_(ak_leq_lub, lub_leq_bk))
    Ik_eq_interval_k = Ik.equals(interval([ak, bk])).by_simplification()  # I[k] = [a[k], b[k]]
    lub_in_Ik = lub_in_interval_k.substitute("left", Ik_eq_interval_k)
    conclude(lub_in_Ik)

intersection = Intersection(I, name="I")

forall_k_lub_in_Ik = ctx.get_proven()[0]
display(lub_A.is_in(intersection).by_predicate(forall_k_lub_in_Ik))


IsContainedIn(lub, Intersection([a_(n),b_(n)]...))

: 

In [3]:
from pylogic import *
settings["PYTHON_OPS_RETURN_PROPS"] = True
a = Sequence("a", real=True); b = Sequence("b", real=True)
n, k0, k1 = map(Variable, ["n","k0", "k1"])
forall_n_in_N_an_leq_bn = ForallInSet(n, Naturals, a[n] <= b[n]).assume()
I = Sequence("I", set_=True, nth_term=lambda n: interval[a[n], b[n]] )

a_increasing_b_decreasing = ForallInSet(
    k0,
    Naturals,
    ForallInSet(
        k1, Naturals, if_(k0 <= k1).then((a[k0] <= a[k1]).and_(b[k1] <= b[k0]))
    ),
).assume()
A = SeqSet(a)

# Part 1: A is nonempty
a0 = a[zero]
a0_in_A = a0.is_in(A).by_inspection()
A_nonempty = a0_in_A.thus_not_empty()

with AssumptionsContext() as ctx:
    k = ctx_var("k", natural=True)
    bk = b[k]; ak = a[k]; Ik = I[k]
    # Part 2: bk is an upper bound for set A
    with AssumptionsContext() as x_ctx:
        x = ctx_var("x")
        x_in_A = x.is_in(A).assume()
        n, (_, x_eq_an) = x_in_A.thus_predicate().rename_variable("n")
        # case 1: n <= k
        with AssumptionsContext() as case_1:
            n_leq_k = assume(n <= k)
            an_leq_ak = a_increasing_b_decreasing(n, k)(n_leq_k)[0]
            ak_leq_bk = forall_n_in_N_an_leq_bn(k)
            conclude(an_leq_ak.transitive(ak_leq_bk))
        impl1 = case_1.get_proven()[0]
        # case 2: k <= n
        with AssumptionsContext() as case_2:
            k_leq_n = assume(k <= n)
            an_leq_bn = forall_n_in_N_an_leq_bn(n)
            bn_leq_bk = a_increasing_b_decreasing(k, n)(k_leq_n)[1]
            conclude(an_leq_bn.transitive(bn_leq_bk))
        impl2 = case_2.get_proven()[0]
        leq_is_strongly_connected = Naturals.order_is_strongly_connected(n, k)
        an_leq_bk = leq_is_strongly_connected.by_cases(impl1, impl2)
        conclude(an_leq_bk.substitute("left", x_eq_an))
    forall_x_in_A_x_leq_bk = x_ctx.get_proven()[0]
    exists_ub_for_A = forall_x_in_A_x_leq_bk.thus_there_exists("ub", bk, Reals)

    # Part 3: A has a least upper bound
    A_has_lub = Reals.theorems.completeness(A)(A_nonempty, exists_ub_for_A).rename_variable("s")
    lub_A, lub_A_is_ub_and_least = A_has_lub.extract()
    lub_A_is_real = lub_A_is_ub_and_least[0]
    lub_A_is_ub = lub_A_is_ub_and_least[1]
    lub_A_is_least = lub_A_is_ub_and_least[2]

    # Part 4: a[k] <= lub_A <= b[k]
    ak_leq_lub = lub_A_is_ub(ak)
    lub_leq_bk = forall_x_in_A_x_leq_bk.modus_ponens(lub_A_is_least(bk))
    lub_in_interval_k = lub_A.is_in(interval[ak, bk]).by_predicate(lub_A_is_real.and_(ak_leq_lub, lub_leq_bk))
    Ik_eq_interval_k = Ik.equals(interval[ak, bk]).by_simplification()  # I[k] = [a[k], b[k]]
    lub_in_Ik = lub_in_interval_k.substitute("left", Ik_eq_interval_k)
    conclude(lub_in_Ik)
forall_k_lub_in_Ik = ctx.get_proven()[0]
intersection = Intersection(I, name="I")
print(lub_A.is_in(intersection).by_predicate(forall_k_lub_in_Ik)._latex())

s \in \bigcap \left(\left[{a}_{n}, {b}_{n}\right]\cdots\right)


In [7]:
eps, delta, x, a, fx, fa = map(Variable, [r"\epsilon", r"\delta", "x", "a", "f(x)", "f(a)"])
ForallInSet(
    eps,
    Reals,
    Implies(
    GreaterThan(eps, 0),
        ExistsInSet(
            delta,
            Reals,
            GreaterThan(delta, 0).and_(
                Implies(LessThan(Abs(x - a), delta), LessThan(Abs(fx - fa), eps))
            ),
        ),
    ),
)

ForallInSet(Variable(\epsilon, deps=()), Set_Reals, Implies(\epsilon > 0, ExistsInSet(Variable(\delta, deps=()), Set_Reals, And(\delta > 0, Implies(LessThan(|x + -a|, \delta), LessThan(|f(x) + -f(a)|, \epsilon))))))